In [18]:
# Swing Trade Algorithm

In [19]:
!pip install alpaca-trade-api

In [20]:
#Import Required Libraries

import os
import requests
import pandas as pd
import hvplot.pandas
import numpy as np
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from pathlib import Path


In [21]:
#Load .env environment variables

# load_dotenv()

In [22]:
# Set Alpaca API Key 

# alpaca_api_key = os.getenv('ALPACA_API_KEY')
#alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')


In [23]:
# Creae the API object

#alpaca = tradeapi.REST(
   # alpaca_api_key,
   # alpaca_secret_key,
   # api_version='V2')

    

In [24]:
# Format current date as ISO format

#start_date = pd.Timestamp('2010-01-01', tz='America/New_York').isoformat()
#end_date= pd.Timestamp('2021-09-01', tz='America/New_York').isoformat()


# Set the tickers 

#SPY = ['SPY']
#QQQ = ['QQQ']
#AAPL = ['AAPL']
#NVDA = ['NVDA']
#TWTR = ['TWTR']

    

In [25]:
# Set timeframe to one day ('1D') for the Alpaca API
#timeframe = '15Min'

# Get current closing prices for SPY

#SPY_df = alpaca.get_barset(
  #  SPY,
  #  timeframe,
  #  start = start_date,
   # end = end_date,
   # limit=1000
#).df

# Display SPY df 

#display(SPY_df.head())
#display(SPY_df.tail())

In [26]:
# Read in the CSV data for SPY

SPY_df = pd.read_csv(Path('./Resources/SPY_Close.csv'),
                     index_col='Date',
                     parse_dates=True,
                     infer_datetime_format=True)

SPY_df.head()

,Open,Close
Date,,
1993-01-29,43.96875,43.93750
1993-02-01,43.96875,44.25000
1993-02-02,44.21875,44.34375
1993-02-03,44.40625,44.81250
1993-02-04,44.96875,45.00000


In [27]:
# Create df for SPY closing prices

SPY_close = pd.DataFrame()

SPY_close['close'] = SPY_df['Close']

# Drop the time component of the date

# SPY_close.index = SPY_close.index.time

# Review SPY DF

display(SPY_close.head())
display(SPY_close.tail())


,close
Date,
1993-01-29,43.93750
1993-02-01,44.25000
1993-02-02,44.34375
1993-02-03,44.81250
1993-02-04,45.00000


,close
Date,
2021-09-03,453.079987
2021-09-07,451.459991
2021-09-08,450.910004
2021-09-09,448.980011
2021-09-10,445.440002


In [28]:
SPY_signals = pd.DataFrame()


SPY_signals['close'] = SPY_close['close']
SPY_signals['returns'] = SPY_close['close'].pct_change()

SPY_signals = SPY_signals.dropna()

SPY_signals.head()

,close,returns
Date,,
1993-02-01,44.25000,0.007112
1993-02-02,44.34375,0.002119
1993-02-03,44.81250,0.010571
1993-02-04,45.00000,0.004184
1993-02-05,44.96875,-0.000694


In [41]:
# Set the short window and long window
short_window = 3
long_window = 9

# Generate the fast and slow simple moving averages 
SPY_signals['SMA_Fast'] = SPY_signals['close'].rolling(window=short_window).mean()
SPY_signals['SMA_Slow'] = SPY_signals['close'].rolling(window=long_window).mean()

SPY_signals = SPY_signals.dropna()

# Review the DataFrame
display(SPY_signals.head())
display(SPY_signals.tail())

SPY_signals["signal"] = 0.0

,close,returns,SMA_Fast,SMA_Slow,signal,strategy returns
Date,,,,,,
1993-03-24,44.87500,0.000000,44.843750,45.069444,0.0,0.000000
1993-03-25,45.15625,0.006267,44.968750,45.076389,0.0,0.000000
1993-03-26,44.90625,-0.005536,44.979167,45.031250,1.0,-0.000000
1993-03-29,45.09375,0.004175,45.052083,45.006944,1.0,0.004175
1993-03-30,45.21875,0.002772,45.072917,45.027778,1.0,0.002772


,close,returns,SMA_Fast,SMA_Slow,signal,strategy returns
Date,,,,,,
2021-09-03,453.079987,-0.000243,452.689992,450.583333,0.0,-0.0
2021-09-07,451.459991,-0.003576,452.576660,450.971110,0.0,-0.0
2021-09-08,450.910004,-0.001218,451.816661,451.193332,0.0,-0.0
2021-09-09,448.980011,-0.004280,450.450002,451.495555,0.0,-0.0
2021-09-10,445.440002,-0.007885,448.443339,450.961110,0.0,-0.0


In [42]:
# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) greater than the long-window (SMA100)
# and 0 is when the condition is not met
SPY_signals["signal"][short_window:] = np.where(
    SPY_signals["SMA_Slow"][short_window:] > SPY_signals["SMA_Fast"][short_window:], 1.0, 0.0
)

# Review the DataFrame
SPY_signals.tail(10)

,close,returns,SMA_Fast,SMA_Slow,signal,strategy returns
Date,,,,,,
2021-08-27,450.250000,0.008941,448.473338,445.232222,0.0,0.0
2021-08-30,452.230011,0.004398,449.580007,446.142222,0.0,0.0
2021-08-31,451.559998,-0.001482,451.346670,447.517778,0.0,-0.0
2021-09-01,451.799988,0.000531,451.863332,448.844445,0.0,0.0
2021-09-02,453.190002,0.003077,452.183329,449.936669,0.0,0.0
2021-09-03,453.079987,-0.000243,452.689992,450.583333,0.0,-0.0
2021-09-07,451.459991,-0.003576,452.576660,450.971110,0.0,-0.0
2021-09-08,450.910004,-0.001218,451.816661,451.193332,0.0,-0.0
2021-09-09,448.980011,-0.004280,450.450002,451.495555,1.0,-0.0


In [43]:
# Initialize the new Signal column
# SPY_signals['signal'] = 0.0

# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
#SPY_signals.loc[(SPY_signals['returns'] >= 0), 'signal'] = 1

# When Actual Returns are less than 0, generate signal to hold 
# SPY_signals.loc[(SPY_signals['returns'] < 0), 'signal'] = 1

# Review the DataFrame
#display(SPY_signals.head())
# display(SPY_signals.tail())

In [44]:
# Initiate the value count

SPY_signals['signal'].value_counts()

0.0    4261
1.0    2909
Name: signal, dtype: int64

In [45]:
# Calculate the strategy returns and add them to the signals_df DataFrame
SPY_signals['strategy returns'] = SPY_signals['returns'] * SPY_signals['signal'].shift()

# Review the DataFrame
display(SPY_signals.head())
display(SPY_signals.tail())

,close,returns,SMA_Fast,SMA_Slow,signal,strategy returns
Date,,,,,,
1993-03-24,44.87500,0.000000,44.843750,45.069444,0.0,NaN
1993-03-25,45.15625,0.006267,44.968750,45.076389,0.0,0.0
1993-03-26,44.90625,-0.005536,44.979167,45.031250,0.0,-0.0
1993-03-29,45.09375,0.004175,45.052083,45.006944,0.0,0.0
1993-03-30,45.21875,0.002772,45.072917,45.027778,0.0,0.0


,close,returns,SMA_Fast,SMA_Slow,signal,strategy returns
Date,,,,,,
2021-09-03,453.079987,-0.000243,452.689992,450.583333,0.0,-0.000000
2021-09-07,451.459991,-0.003576,452.576660,450.971110,0.0,-0.000000
2021-09-08,450.910004,-0.001218,451.816661,451.193332,0.0,-0.000000
2021-09-09,448.980011,-0.004280,450.450002,451.495555,1.0,-0.000000
2021-09-10,445.440002,-0.007885,448.443339,450.961110,1.0,-0.007885


In [46]:
(1 + SPY_signals[['strategy returns','returns']]).cumprod().hvplot()

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

---

### Generating the Features and Target Sets

#### Create the features set

In [61]:
# Assign a copy of the sma_fast and sma_slow columns to a new DataFrame called X
X = SPY_signals[['SMA_Fast', 'SMA_Slow']].copy()

# Display sample data
display(X.head())
display(X.tail())

,SMA_Fast,SMA_Slow
time,,
2021-07-19 16:15:00-04:00,424.920000,423.424444
2021-07-19 16:30:00-04:00,425.150000,423.647778
2021-07-20 08:00:00-04:00,425.743333,424.167778
2021-07-20 08:15:00-04:00,426.256667,424.643333
2021-07-20 08:30:00-04:00,426.620000,425.018889


,SMA_Fast,SMA_Slow
time,,
2021-08-31 15:30:00-04:00,451.481667,451.628889
2021-08-31 15:45:00-04:00,451.540000,451.619444
2021-08-31 16:00:00-04:00,451.856667,451.660556
2021-08-31 16:15:00-04:00,452.030000,451.725556
2021-08-31 16:45:00-04:00,452.300000,451.816667


#### Create the target set

In [62]:
# Create a new column in the SPY_signals called signal setting its value to zero.
SPY_signals['signal'] = 0.0

In [63]:
# Create the signal to buy
SPY_signals.loc[(SPY_signals['returns'] >= 0), 'signal'] = 1

In [64]:
# Create the signal to sell
SPY_signals.loc[(SPY_signals['returns'] < 0), 'signal'] = -1

In [65]:
# Copy the new signal column to a new Series called y.
y = SPY_signals['signal'].copy()

display(y.head())
display(y.tail())

time
2021-07-19 16:15:00-04:00    1.0
2021-07-19 16:30:00-04:00    1.0
2021-07-20 08:00:00-04:00    1.0
2021-07-20 08:15:00-04:00   -1.0
2021-07-20 08:30:00-04:00   -1.0
Name: signal, dtype: float64

time
2021-08-31 15:30:00-04:00    1.0
2021-08-31 15:45:00-04:00   -1.0
2021-08-31 16:00:00-04:00    1.0
2021-08-31 16:15:00-04:00    1.0
2021-08-31 16:45:00-04:00    1.0
Name: signal, dtype: float64

---

### Split the Data Into Training and Testing Datasets

#### Creating the Training Datasets

In [66]:
# Imports 
from pandas.tseries.offsets import DateOffset

In [67]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2021-07-19 16:15:00-04:00


In [68]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=1)

# Display the training end date
print(training_end)

2021-08-19 16:15:00-04:00


In [69]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Display sample data
X_train.head()

,SMA_Fast,SMA_Slow
time,,
2021-07-19 16:15:00-04:00,424.920000,423.424444
2021-07-19 16:30:00-04:00,425.150000,423.647778
2021-07-20 08:00:00-04:00,425.743333,424.167778
2021-07-20 08:15:00-04:00,426.256667,424.643333
2021-07-20 08:30:00-04:00,426.620000,425.018889


In [70]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Display sample data
X_test.head()

,SMA_Fast,SMA_Slow
time,,
2021-08-19 16:15:00-04:00,439.996667,439.710000
2021-08-19 16:30:00-04:00,440.003333,439.908889
2021-08-19 16:45:00-04:00,439.970000,440.056111
2021-08-20 08:30:00-04:00,439.626667,439.975000
2021-08-20 09:00:00-04:00,439.726667,440.000556


#### Standardizing the Data

In [71]:
# Imports
from sklearn.preprocessing import StandardScaler

In [72]:
# Create a StandardScaler instance
scaler = StandardScaler()
 
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
 
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

### Incorporate the SVM Machine Learning Into the Trading Strategy

In [73]:
# Imports
from sklearn import svm
from sklearn.metrics import classification_report

In [74]:
# Create the classifier model
svm_model = svm.SVC()
 
# Fit the model to the data using X_train_scaled and y_train
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data
training_signal_predictions = svm_model.predict(X_train_scaled)

# Display the sample predictions
training_signal_predictions[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [75]:
# Evaluate the model using a classification report
training_report = classification_report(y_train, training_signal_predictions)
print(training_report)

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00       332
         1.0       0.55      1.00      0.71       405

    accuracy                           0.55       737
   macro avg       0.27      0.50      0.35       737
weighted avg       0.30      0.55      0.39       737



/Users/martiquehenton/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/martiquehenton/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/martiquehenton/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

---

### Backtesting a Machine Learning Trading Algorithm

In [76]:
# Use the trained model to predict the trading signals for the testing data.
testing_signal_predictions = svm_model.predict(X_test_scaled)

In [77]:
# Evaluate the model's ability to predict the trading signal for the testing data
svm_testing_report = classification_report(y_test, testing_signal_predictions)
print(svm_testing_report)

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00       106
         1.0       0.58      1.00      0.74       149

    accuracy                           0.58       255
   macro avg       0.29      0.50      0.37       255
weighted avg       0.34      0.58      0.43       255



/Users/martiquehenton/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/martiquehenton/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/martiquehenton/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [78]:
# Create a new empty predictions DataFrame using code provided below.
predictions_df = pd.DataFrame(index=X_test.index)
predictions_df["predicted_signal"] = testing_signal_predictions
predictions_df["actual_returns"] = SPY_signals["returns"]
predictions_df["trading_algorithm_returns"] = predictions_df["actual_returns"] * predictions_df["predicted_signal"]
predictions_df.head()

,predicted_signal,actual_returns,trading_algorithm_returns
time,,,
2021-08-19 16:15:00-04:00,1.0,-0.000341,-0.000341
2021-08-19 16:30:00-04:00,1.0,0.000091,0.000091
2021-08-19 16:45:00-04:00,1.0,0.000023,0.000023
2021-08-20 08:30:00-04:00,1.0,-0.002455,-0.002455
2021-08-20 09:00:00-04:00,1.0,0.003121,0.003121


In [79]:
# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
(1 + predictions_df[["actual_returns", "trading_algorithm_returns"]]).cumprod().plot()

<AxesSubplot:xlabel='time'>

---

# Add a New Machine Learning Model

## Step 2: Import the `LogisticRegression` model from scikit-learn.


In [80]:
# Import LogisticRegression from sklearn
from sklearn.linear_model import LogisticRegression

In [88]:
# Create an instance of the LogisticRegression model
logistic_regression_model = LogisticRegression()

## Step 3: Using the same training data that the SVM model used (`X_train_scaled` and `y_train`), fit the `LogisticRegression` model.

In [89]:
# Fit the LogisticRegression model
logistic_regression_model.fit(X_train_scaled, y_train)

LogisticRegression()

## Step 4: Use the trained model to predict the trading signals for the training data. Use the `classification_report` module to evaluate the model.

In [90]:
# Use the trained LogisticRegression model to predict the trading signals for the training data
lr_training_signal_predictions = logistic_regression_model.predict(X_train_scaled)

# Diplay the predictions
lr_training_signal_predictions

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1.,
       -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1

In [91]:
# Generate a classification report using the training data and the logistic regression model's predications
lr_training_report = classification_report(y_train, lr_training_signal_predictions)

# Review the classification report
print(lr_training_report)

              precision    recall  f1-score   support

        -1.0       0.60      0.08      0.14       332
         1.0       0.56      0.96      0.71       405

    accuracy                           0.56       737
   macro avg       0.58      0.52      0.42       737
weighted avg       0.58      0.56      0.45       737



## Step 5: Backtest the `LogisticRegression` model using the testing data to evaluate its performance.


In [92]:
# Use the trained model to predict the trading signals for the testing data.
lr_testing_signal_predictions = logistic_regression_model.predict(X_test_scaled)

In [93]:
# Generate a classification report using the testing data and the logistic regression model's predictions
lr_testing_report = classification_report(y_test, lr_testing_signal_predictions)

# Review the testing classification report
print(lr_testing_report)

              precision    recall  f1-score   support

        -1.0       0.42      0.05      0.08       106
         1.0       0.58      0.95      0.72       149

    accuracy                           0.58       255
   macro avg       0.50      0.50      0.40       255
weighted avg       0.51      0.58      0.46       255



## Step 6: Compare the performance of the logistic regression and SVM models using the classification reports generated with the testing data. 

In [94]:
# Print the classification report for the Logistic Regression model using the test data
print("Logistic Regression Classification Report")
print(lr_testing_report)

Logistic Regression Classification Report
              precision    recall  f1-score   support

        -1.0       0.42      0.05      0.08       106
         1.0       0.58      0.95      0.72       149

    accuracy                           0.58       255
   macro avg       0.50      0.50      0.40       255
weighted avg       0.51      0.58      0.46       255



In [95]:
# Print the classification report for the SVM model using the test data
print("SVM Classification Report")
print(svm_testing_report)

SVM Classification Report
              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00       106
         1.0       0.58      1.00      0.74       149

    accuracy                           0.58       255
   macro avg       0.29      0.50      0.37       255
weighted avg       0.34      0.58      0.43       255

